In [1]:
import nest_asyncio
nest_asyncio.apply()
del nest_asyncio

In [2]:
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)

Building: found in cache, done.
Messages from stanc:
Warning in '/var/folders/79/lndg0b5n2q3b40m_ty2kxlm40000gn/T/httpstan_f6j1s3o1/model_qv763guo.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/var/folders/79/lndg0b5n2q3b40m_ty2kxlm40000gn/T/httpstan_f6j1s3o1/model_qv763guo.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [3]:
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas
print(df.describe().T)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 5.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.54 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.1 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!


                count       mean       std           min        25%  \
parameters                                                            
lp__           4000.0 -39.534598  2.614211 -5.284539e+01 -41.072309   
accept_stat__  4000.0   0.883403  0.188645  1.471249e-89   0.875860   
stepsize__     4000.0   0.314505  0.030040  2.643172e-01   0.308836   
treedepth__    4000.0   3.615000  0.520906  1.000000e+00   3.000000   
n_leapfrog__   4000.0  13.201500  3.633414  1.000000e+00  15.000000   
divergent__    4000.0   0.000000  0.000000  0.000000e+00   0.000000   
energy__       4000.0  44.570153  3.433271  3.552232e+01  42.106609   
mu             4000.0   8.084615  5.153346 -1.475586e+01   4.737535   
tau            4000.0   6.696274  5.846215  1.064820e-02   2.592279   
eta.1          4000.0   0.369960  0.930824 -3.578056e+00  -0.223366   
eta.2          4000.0   0.007623  0.862447 -2.918535e+00  -0.555868   
eta.3          4000.0  -0.199224  0.940796 -4.105981e+00  -0.833550   
eta.4 